<a href="https://colab.research.google.com/github/hiteshisawariya485/DeepLearning/blob/main/practical_12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler

import warnings
warnings.filterwarnings("ignore")

data= pd.read_csv("seattle-weather.csv")
data.dropna(inplace=True)
data

data.duplicated().sum()

data['date'] = pd.to_datetime(data['date'])

import plotly.express as px
# Creating the line plot using plotly.express
fig = px.line(data, x='date', y=['temp_max', 'temp_min'],
              labels={'value': 'Temperature (°C)', 'date': 'Date'},
              title='Daily Max and Min Temperatures (2012-2015)')

fig.for_each_trace(lambda t: t.update(name={'temp_max': 'Max Temp', 'temp_min': 'Min Temp'}[t.name],
                                          line=dict(color='#636efb' if t.name == 'temp_max' else '#01008c')))
fig.update_layout(
    legend_title_text='Temperature Type',
    xaxis_title='Date',
    yaxis_title='Temperature (°C)',
    template='plotly_white'
)

fig.show()

weather_counts = data['weather'].value_counts().reset_index()
weather_counts.columns = ['Weather Type', 'Count']

# Create a bar chart to display the distribution of weather types
fig = px.bar(weather_counts, x='Weather Type', y='Count',
             title='Distribution of Weather Types',
             labels={'Weather Type': 'Weather Type', 'Count': 'Count'},
             color='Count',
             color_discrete_map={
                 'drizzle': '#0066cc',
                 'rain': '#004080',
                 'fog': '#003366',
                 'sun': '#005bb5',
             })
fig.show()

fig = px.scatter(data, x='temp_max', y='precipitation',
                 title='Precipitation vs. Max Temperature',
                 labels={'temp_max': 'Max Temperature (°C)', 'precipitation': 'Precipitation (mm)'})

fig.update_layout(width=800, height=500)
fig.show()

fig = px.scatter_matrix(data,
                        dimensions=['precipitation', 'temp_max', 'temp_min', 'wind'],
                        color='weather',
                        title='Pairplot of Weather Data by Weather Type',
                        labels={'weather': 'Weather Type'},
                        height=1000, width=1000)

fig.update_traces()
fig.show()

#coulmn Open converted into numpy array
training = data.iloc[:,2:3].values

len(training)

def df_to_XY(df,window_size=10):
 X_train=[]
 y_train=[]

 for i in range(10,len(training)):
    X_train.append(training[i-10:i,0])
    y_train.append(training[i,0])

 X_train, y_train = np.array(X_train), np.array(y_train)
 return X_train, y_train

WINDOW = 10
X,y = df_to_XY(data,WINDOW)
print(len(X),len(y))
X_train = X[:800]
y_train = y[:800]
X_val = X[800:1000]
y_val = y[800:1000]
X_test = X[1000:]
y_test = y[1000:]

X_train = np.reshape(X_train,(X_train.shape[0],X_train.shape[1],1))
X_val = np.reshape(X_val,(X_val.shape[0],X_val.shape[1],1))
X_test = np.reshape(X_test,(X_test.shape[0],X_test.shape[1],1))

from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
model = Sequential()
#Addinf the first LSTM layer and some Dropout regularisation
model.add(LSTM(units=50, return_sequences = True, input_shape=(X_train.shape[1], 1)))
model.add(Dropout(0.2))

model.add(LSTM(units=50, return_sequences = True))
model.add(Dropout(0.2))

model.add(LSTM(units=50, return_sequences = True))
model.add(Dropout(0.2))

model.add(LSTM(units=50))
model.add(Dropout(0.2))
#output layer
model.add(Dense(units=1))

#Compiling
model.compile(optimizer='adam',loss='mean_squared_error')

from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import Adam

history=model.fit(X_train,y_train,validation_data=(X_val,y_val),epochs=100, batch_size=32)

hist = pd.DataFrame(history.history)

history_loss = hist[['loss', 'val_loss']].reset_index()

fig = px.line(history_loss,
              x=history_loss.index,
              y=['loss', 'val_loss'],
              labels={'index': 'Epochs', 'value': 'Loss'},
              title='Loss & Val Loss')
fig.update_layout(width=700, height=500)

fig.show()

train_pred = model.predict(X_train).flatten()
val_pred = model.predict(X_val).flatten()
test_pred = model.predict(X_test).flatten()

# Show the validation results plot
pred = np.concatenate([train_pred, val_pred, test_pred])
df_pred = pd.DataFrame(data["temp_max"].copy())
df_pred.columns = ["actual"]
df_pred = df_pred[WINDOW:]
df_pred["predicted"] = pred

df_long = df_pred[800:].reset_index()
df_long = df_long.melt(id_vars='index', value_vars=['actual', 'predicted'],
                        var_name='Type', value_name='Temperature')

fig_val = px.line(df_long, x='index', y='Temperature', color='Type',
                  title="Validation Results",
                  labels={'Temperature': 'Temperature', 'index': 'Time'},
                  color_discrete_map={'actual': 'blue', 'predicted': 'red'})
fig.update_layout(width=700, height=500)

fig_val.show()

# Show the test results plot
fig_test = px.line(df_pred[1000:], title="Test Results",
                   labels={'value': 'Temperature', 'index': 'Time'},
                   line_shape='linear')
fig_test.update_traces(name='Actual', line_color='blue')
fig_test.add_scatter(x=df_pred.index[1000:], y=df_pred['predicted'][1000:],
                      mode='lines', name='Predicted', line=dict(color='red'))
fig.update_layout(width=700, height=500)
fig_test.show()

from sklearn.metrics import mean_squared_error
train_mse = mean_squared_error(y_train, train_pred)
val_mse = mean_squared_error(y_val, val_pred)
test_mse = mean_squared_error(y_test, test_pred)
print(f"Train MSE: {train_mse:.4f}")
print(f"Validation MSE: {val_mse:.4f}")
print(f"Test MSE: {test_mse:.4f}")

1451 1451
Epoch 1/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 8s 78ms/step - loss: 247.1539 - val_loss: 234.4637
Epoch 2/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 37ms/step - loss: 99.5529 - val_loss: 155.0164
Epoch 3/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - loss: 74.8861 - val_loss: 128.4997
Epoch 4/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - loss: 66.4535 - val_loss: 110.2364
Epoch 5/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - loss: 61.5495 - val_loss: 98.0846
Epoch 6/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - loss: 55.5326 - val_loss: 89.3511
Epoch 7/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - loss: 55.0542 - val_loss: 84.2828
Epoch 8/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - loss: 57.4659 - val_loss: 81.0439
Epoch 9/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - loss: 51.8926 - val_loss: 79.5715
Epoch 10/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - loss: 53.1236 - val_loss: 77.5204
Epoch 11/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - loss: 55.7229 - val_loss: 75.7467
Epoch 12/100
25

25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 109ms/step
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step


Train MSE: 6.9193
Validation MSE: 9.7585
Test MSE: 8.7542
